<a href="https://colab.research.google.com/github/Yallbethel/Beta5/blob/main/Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Yallbethel/Beta5.git



Cloning into 'Beta5'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 28 (delta 5), reused 28 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 8.42 MiB | 20.00 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
!apt-get install -y flex bison git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
The following additional packages will be installed:
  libfl-dev libfl2
Suggested packages:
  bison-doc flex-doc
The following NEW packages will be installed:
  bison flex libfl-dev libfl2
0 upgraded, 4 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,072 kB of archives.
After this operation, 3,667 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 flex amd64 2.6.4-8build2 [307 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 bison amd64 2:3.8.2+dfsg-1build1 [748 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfl2 amd64 2.6.4-8build2 [10.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfl-dev amd64 2.6.4-8build2 [6,236 B]
Fetched 1,072 kB in 0s (4,150 kB/s)
Selecting previously unselected package flex.
(Reading database ... 126209 

In [ ]:
#  Create the Lex program file
%%writefile text_analyzer.l
%{
#include <stdio.h>
int chars = 0;
int words = 0;
int spaces = 0;
int lines = 0;
%}

%%
[a-zA-Z0-9]+   { words++; chars += yyleng; }  // Words (alphanumeric sequences)
[ \t]          { spaces++; chars++; }        // Spaces and tabs
\n             { lines++; chars++; }         // Newlines
.              { chars++; }                  // All other characters
%%

int yywrap() { return 1; }

int main(int argc, char *argv[]) {
    if (argc > 1) {
        yyin = fopen(argv[1], "r");
        if (!yyin) {
            fprintf(stderr, "Error: Could not open %s\n", argv[1]);
            return 1;
        }
    }

    yylex(); // Run the lexer

    printf("\n=== TEXT ANALYSIS RESULTS ===\n");
    printf("Total characters: %d\n", chars);
    printf("Total words: %d\n", words);
    printf("Total spaces: %d\n", spaces);
    printf("Total lines: %d\n", lines);

    if (argc > 1) fclose(yyin);
    return 0;
}

Overwriting text_analyzer.l


In [ ]:
#  Compile the program
!flex text_analyzer.l
!gcc lex.yy.c -o text_analyzer -lfl


In [ ]:
# Create a test file
%%writefile test.txt
This is a sample text file.
It contains 3 lines of text,
with  varying  spacing.

Numbers like 123 are included!


Writing test.txt


In [ ]:
# Run the analysis
!./text_analyzer test.txt


=== TEXT ANALYSIS RESULTS ===
Total characters: 113
Total words: 20
Total spaces: 18
Total lines: 5


In [ ]:
%%writefile email.l
%{
#include <stdio.h>
#include <string.h>
void validate_email(char *email);
%}

%%
[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,} {
    printf("VALID: %s\n", yytext);
}
.+ {
    printf("INVALID: %s - : ", yytext);
    validate_email(yytext);
}
%%

void validate_email(char *email) {
    int at_count = 0;
    char *ptr = email;

    // Check for '@'
    while (*ptr) {
        if (*ptr == '@') at_count++;
        ptr++;
    }
    if (at_count == 0) {
        printf("Missing '@'\n");
        return;
    } else if (at_count > 1) {
        printf("Multiple '@' symbols\n");
        return;
    }

    // Check domain format (e.g., "domain.com")
    char *dot = strrchr(email, '.');
    if (!dot || dot == email || strlen(dot) < 3) {
        printf("Invalid domain (e.g., must end with '.com')\n");
        return;
    }

    printf("General syntax error\n");
}

int main() {
    yylex();
    return 0;
}

int yywrap() {
    return 1;
}

Writing email.l


In [ ]:
!flex email.l
!gcc lex.yy.c -o email -ll

In [ ]:
%%file emails.txt
valid.email@example.com
invalid@.com
missing.at.sign
another.valid@test.co.uk
@missing.local.com
dot.before@.domain.com
short@a.a
valid+alias@domain.org

Writing emails.txt


In [ ]:
!./email < emails.txt

VALID: valid.email@example.com

INVALID: invalid@.com - : General syntax error

INVALID: missing.at.sign - : Missing '@'

VALID: another.valid@test.co.uk

INVALID: @missing.local.com - : General syntax error

VALID: dot.before@.domain.com

INVALID: short@a.a - : Invalid domain (e.g., must end with '.com')

VALID: valid+alias@domain.org



In [ ]:
%%writefile password.l
%{
#include <stdio.h>
#include <string.h>
#include <stdbool.h>

bool has_upper = false;
bool has_lower = false;
bool has_digit = false;
bool has_special = false;
%}

%%
^.{9,}$ {
    // Check each character in the password
    for(int i = 0; i < strlen(yytext); i++) {
        if(yytext[i] >= 'A' && yytext[i] <= 'Z') has_upper = true;
        else if(yytext[i] >= 'a' && yytext[i] <= 'z') has_lower = true;
        else if(yytext[i] >= '0' && yytext[i] <= '9') has_digit = true;
        else if(strchr("!@#$%^&*()_+-=[]{}|;':\",./<>?`~", yytext[i])) has_special = true;
    }

    if(has_upper && has_lower && has_digit && has_special) {
        printf("Valid password: %s\n", yytext);
    } else {
        printf("Invalid password: %s - Missing requirements:\n", yytext);
        if(!has_upper) printf("  * At least one uppercase letter\n");
        if(!has_lower) printf("  * At least one lowercase letter\n");
        if(!has_digit) printf("  * At least one digit\n");
        if(!has_special) printf("  * At least one special character\n");
    }

    // Reset flags for next password
    has_upper = has_lower = has_digit = has_special = false;
}

.+ {
    printf("Invalid password: %s - Must be longer than 8 characters\n", yytext);
}
%%

int main(int argc, char **argv) {
    if(argc > 1) {
        FILE *file = fopen(argv[1], "r");
        if(file) {
            yyin = file;
        } else {
            printf("Could not open file %s\n", argv[1]);
            return 1;
        }
    } else {
        printf("Enter passwords to validate (one per line, Ctrl+D to end):\n");
    }

    yylex();

    if(argc > 1) {
        fclose(yyin);
    }

    return 0;
}

int yywrap() {
    return 1;
}

Writing password.l


In [ ]:
!flex password.l

In [ ]:
!gcc lex.yy.c -o password_validator -lfl

In [ ]:
%%file test_passwords.txt
Password1!
short
NoDigitHere!
NOLOWER1!
noupper1!
ValidPass123!
AnotherGood1@

Writing test_passwords.txt


In [ ]:
!./password_validator test_passwords.txt

Valid password: Password1!

Invalid password: short - Must be longer than 8 characters

Invalid password: NoDigitHere! - Missing requirements:
  * At least one digit

Invalid password: NOLOWER1! - Missing requirements:
  * At least one lowercase letter

Invalid password: noupper1! - Missing requirements:
  * At least one uppercase letter

Valid password: ValidPass123!

Valid password: AnotherGood1@



In [ ]:
# Create the lexer file (calc.l)
%%writefile calc.l
%{
#include "calc.tab.h"
#include <stdio.h>
void yyerror(char *);
extern int yylval;
%}

%%
[0-9]+          { yylval = atoi(yytext); return NUMBER; }
[ \t]           ; /* ignore whitespace */
\n              { return EOL; }
"+"             { return ADD; }
"-"             { return SUB; }
"*"             { return MUL; }
"/"             { return DIV; }
"("             { return LPAREN; }
")"             { return RPAREN; }
.               { yyerror("Invalid character"); }
%%

int yywrap(void) { return 1; }

Writing calc.l


In [ ]:
# Create the parser file (calc.y)
%%writefile calc.y
%{
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

void yyerror(char *);
int yylex(void);
extern int yylval;
%}

%token NUMBER
%token ADD SUB MUL DIV
%token LPAREN RPAREN
%token EOL

%left ADD SUB
%left MUL DIV

%%
input:    /* empty */
        | input line
        ;

line:     EOL
        | expr EOL   { printf(" = %d\n", $1); }
        | error EOL  { yyerror("Error"); }
        ;

expr:     NUMBER           { $$ = $1; }
        | expr ADD expr   { $$ = $1 + $3; }
        | expr SUB expr   { $$ = $1 - $3; }
        | expr MUL expr   { $$ = $1 * $3; }
        | expr DIV expr   { if ($3 == 0) yyerror("Division by zero!");
                           else $$ = $1 / $3; }
        | LPAREN expr RPAREN { $$ = $2; }
        ;
%%

void yyerror(char *s) {
    fprintf(stderr, "Error: %s\n", s);
}

int main(void) {
    printf("Simple Calculator - Enter expressions:\n");
    yyparse();
    return 0;
}

Writing calc.y


In [ ]:
# COMPILE ALL TOGETHER
!bison -d calc.y
!flex calc.l
!gcc calc.tab.c lex.yy.c -o calc -lm

In [ ]:
# TEST THE CALCULATOR
print("\nTESTING THE CALCULATOR:")
!echo "3 + 4 * 2" | ./calc
!echo "(3 + 4) * 2" | ./calc
!echo "10 / 3" | ./calc
!echo "10 / 0" | ./calc


TESTING THE CALCULATOR:
Simple Calculator - Enter expressions:
 = 11
Simple Calculator - Enter expressions:
 = 14
Simple Calculator - Enter expressions:
 = 3
Simple Calculator - Enter expressions:
Error: Division by zero!
 = 10


In [ ]:

%%writefile url_validator.l
%{
#include <stdio.h>
#include <string.h>
#include <stdbool.h>

void validate_and_explain(const char *url);
%}

%%
^https?:\/\/[a-zA-Z0-9\-]+(\.[a-zA-Z0-9\-]+)+(\/[a-zA-Z0-9\-._~:\/?%&=]*)?$ { printf("✅ VALID URL: %s\n", yytext); }
^ftp:\/\/[a-zA-Z0-9\-]+(\.[a-zA-Z0-9\-]+)+(\/[a-zA-Z0-9\-._~:\/?%&=]*)?$    { printf("✅ VALID URL: %s\n", yytext); }
^[a-zA-Z0-9\-]+(\.[a-zA-Z0-9\-]+)+(\/[a-zA-Z0-9\-._~:\/?%&=]*)?$           { printf("⚠️  WARNING: %s (Missing protocol)\n", yytext); }
.+\n                                                                        { validate_and_explain(yytext); }
\n                                                                          ;
%%

int yywrap() { return 1; }

void validate_and_explain(const char *url) {
    bool is_valid = true;
    printf("\n🔍 Analyzing: %s\n", url);

    // Remove newline if present
    char clean_url[256];
    strncpy(clean_url, url, sizeof(clean_url));
    clean_url[strcspn(clean_url, "\n")] = '\0';

    // 1. Check protocol
    if (strstr(clean_url, "http://") == NULL &&
        strstr(clean_url, "https://") == NULL &&
        strstr(clean_url, "ftp://") == NULL) {
        printf("✖ Missing protocol (must start with http://, https:// or ftp://)\n");
        is_valid = false;
    }

    // 2. Check domain structure
    if (strchr(clean_url, '.') == NULL) {
        printf("✖ Invalid domain (must contain at least one dot)\n");
        is_valid = false;
    }

    // 3. Check for invalid characters
    const char *invalid_chars = " !@#$^&*()+=[]{}|;:'\"<>,";
    for (int i = 0; invalid_chars[i]; i++) {
        if (strchr(clean_url, invalid_chars[i])) {
            printf("✖ Contains invalid character: '%c'\n", invalid_chars[i]);
            is_valid = false;
        }
    }

    // 4. Check for spaces
    if (strchr(clean_url, ' ')) {
        printf("✖ Contains spaces (URLs cannot contain spaces)\n");
        is_valid = false;
    }

    if (is_valid) {
        printf("✅ VALID URL\n");
    } else {
        printf("❌ INVALID URL\n");
    }
}

int main() {
    printf("=== URL VALIDATOR ===\n");
    printf("Enter URLs (one per line, Ctrl+D to finish):\n");
    yylex();
    return 0;
}

Writing url_validator.l


In [ ]:

!flex url_validator.l
!gcc lex.yy.c -o validator -lfl



In [ ]:
%%file test_urls.txt
https://www.google.com
http://example.com/path?query=value
ftp://files.example.org
invalid-url
example.com
www.example.org/path/to/page.html
not_a_url
https://sub.domain.co.uk:8080/api/v1
http://bad!char.com
missing dot.com
http://
https://with space.com
http://valid-but-missing.dot
https://valid.com/with#hash
http://valid.com/with?query=param

Writing test_urls.txt


In [ ]:
# Run tests
!echo "=== TESTING URL VALIDATOR ==="
!cat test_urls.txt | ./validator

=== TESTING URL VALIDATOR ===
=== URL VALIDATOR ===
Enter URLs (one per line, Ctrl+D to finish):
✅ VALID URL: https://www.google.com
✅ VALID URL: http://example.com/path?query=value
✅ VALID URL: ftp://files.example.org

🔍 Analyzing: invalid-url

✖ Missing protocol (must start with http://, https:// or ftp://)
✖ Invalid domain (must contain at least one dot)
❌ INVALID URL
⚠️  WARNING: example.com (Missing protocol)
⚠️  WARNING: www.example.org/path/to/page.html (Missing protocol)

🔍 Analyzing: not_a_url

✖ Missing protocol (must start with http://, https:// or ftp://)
✖ Invalid domain (must contain at least one dot)
❌ INVALID URL

🔍 Analyzing: https://sub.domain.co.uk:8080/api/v1

✖ Contains invalid character: ':'
❌ INVALID URL

🔍 Analyzing: http://bad!char.com

✖ Contains invalid character: '!'
✖ Contains invalid character: ':'
❌ INVALID URL

🔍 Analyzing: missing dot.com

✖ Missing protocol (must start with http://, https:// or ftp://)
✖ Contains invalid character: ' '
✖ Contains space

In [ ]:
# Create the corrected Lex program file
%%writefile wdu_id_validator.l
%{
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int current_year = 2017;  // Current year is 2017
%}

%%
^WDU[0-9]{6}$ {
    // Extract the 2-digit batch year (positions 3-4)
    char year_part[3] = {yytext[3], yytext[4], '\0'};
    int batch_year = 2000 + atoi(year_part);

    if (batch_year >= 2007 && batch_year <= 2017) {
        printf("\033[1;32mVALID WDU ID: %s\033[0m\n", yytext);
        printf("  Batch Year: %d\n", batch_year);
        printf("  Student Number: %.4s\n", yytext+5);
    } else if (batch_year < 2007) {
        printf("\033[1;31mINVALID WDU ID: %s\033[0m\n", yytext);
        printf("  Reason: Batch year %d is too old (valid: 2007-2017)\n", batch_year);
    } else {
        printf("\033[1;31mINVALID WDU ID: %s\033[0m\n", yytext);
        printf("  Reason: Batch year %d is in the future (current year: 2017)\n", batch_year);
    }
}
^WDU[^0-9].*$ {
    printf("\033[1;31mINVALID WDU ID: %s\033[0m\n", yytext);
    printf("  Reason: Must contain only digits after WDU\n");
}
^WDU[0-9]{1,5}$ {
    printf("\033[1;31mINVALID WDU ID: %s\033[0m\n", yytext);
    printf("  Reason: Too short - need 6 digits after WDU\n");
}
^WDU[0-9]{7,}$ {
    printf("\033[1;31mINVALID WDU ID: %s\033[0m\n", yytext);
    printf("  Reason: Too long - need exactly 6 digits after WDU\n");
}
^[^W].*$ {
    printf("\033[1;31mINVALID WDU ID: %s\033[0m\n", yytext);
    printf("  Reason: Must start with WDU\n");
}
^W[^D].*$ {
    printf("\033[1;31mINVALID WDU ID: %s\033[0m\n", yytext);
    printf("  Reason: Must start with WDU\n");
}
^WD[^U].*$ {
    printf("\033[1;31mINVALID WDU ID: %s\033[0m\n", yytext);
    printf("  Reason: Must start with WDU\n");
}
.|\n {
    printf("\033[1;31mINVALID WDU ID: %s\033[0m\n", yytext);
    printf("  Reason: Invalid format\n");
}
%%

int yywrap() { return 1; }

int main(int argc, char *argv[]) {
    // Test cases
    char *test_ids[] = {
        "WDU071234",  // Valid (2007)
        "WDU142536",  // Valid (2014)
        "WDU171234",  // Valid (2017)
        "WDU061234",  // Invalid (2006)
        "WDU181234",  // Invalid (2018)
        "WDU201234",  // Invalid (future year)
        "WDU12345",   // Invalid (too short)
        "WDU1234567", // Invalid (too long)
        "XDU142536",  // Invalid (wrong prefix)
        NULL
    };

    printf("Woldia University Student ID Validator\n");
    printf("Current Year: 2017 | Valid Batch Years: 2007-2017\n");
    printf("Valid format: WDUYYXXXX (YY = last 2 digits of batch year)\n\n");

    for (int i = 0; test_ids[i] != NULL; i++) {
        printf("Testing: %s\n", test_ids[i]);
        // Add newline to input string
        char input[20];
        snprintf(input, sizeof(input), "%s\n", test_ids[i]);
        yy_scan_string(input);
        yylex();
        printf("\n");
    }
    return 0;
}

Writing wdu_id_validator.l


In [ ]:
# Compile the program
!flex wdu_id_validator.l
!gcc lex.yy.c -o wdu_id_validator -lfl


In [ ]:
#  Run tests
!./wdu_id_validator

Woldia University Student ID Validator
Current Year: 2017 | Valid Batch Years: 2007-2017
Valid format: WDUYYXXXX (YY = last 2 digits of batch year)

Testing: WDU071234
VALID WDU ID: WDU071234
  Batch Year: 2007
  Student Number: 1234
INVALID WDU ID: 

  Reason: Invalid format

Testing: WDU142536
VALID WDU ID: WDU142536
  Batch Year: 2014
  Student Number: 2536
INVALID WDU ID: 

  Reason: Invalid format

Testing: WDU171234
VALID WDU ID: WDU171234
  Batch Year: 2017
  Student Number: 1234
INVALID WDU ID: 

  Reason: Invalid format

Testing: WDU061234
INVALID WDU ID: WDU061234
  Reason: Batch year 2006 is too old (valid: 2007-2017)
INVALID WDU ID: 

  Reason: Invalid format

Testing: WDU181234
INVALID WDU ID: WDU181234
  Reason: Batch year 2018 is in the future (current year: 2017)
INVALID WDU ID: 

  Reason: Invalid format

Testing: WDU201234
INVALID WDU ID: WDU201234
  Reason: Batch year 2020 is in the future (current year: 2017)
INVALID WDU ID: 

  Reason: Invalid format

Testing: WDU1

In [ ]:
%%writefile language_recognition.l
%{
#include <stdio.h>
%}

%%

[A-Za-z]+                             { printf("English/Latin: %s\n", yytext); }
([\xD0\xD1][\x80-\xBF])+              { printf("Cyrillic: %s\n", yytext); }
([\xD8-\xDB][\x80-\xBF])+             { printf("Arabic: %s\n", yytext); }
([\xE4\xE5][\x80-\xBF])+ | ([\xE6][\x80-\xA4])+  { printf("Chinese: %s\n", yytext); }
([\xE1][\x88-\x8F][\x80-\xBF])+       { printf("Amharic/Ge'ez: %s\n", yytext); }

.|\n { /* Ignore other characters */ }

%%

int main() {
    yylex();
    return 0;
}

int yywrap() {
    return 1;
}


Writing language_recognition.l


In [ ]:
!flex language_recognition.l
!gcc lex.yy.c -o language_recognition

In [ ]:
!echo "Hello ሰላም Привет مرحبا 你好" | ./language_recognition

English/Latin: Hello
Amharic/Ge'ez: ሰላም
Cyrillic: Привет
Arabic: مرحبا
Amharic/Ge'ez: �
Amharic/Ge'ez: �


In [ ]:
%%writefile fidel_orders.l
%{
#include <stdio.h>
%}

%%

(ሀ|ለ|ሐ|መ|ረ|ሰ|ቀ|በ|ተ|ነ|አ|ከ|ወ|ዘ|ደ|ገ|ጠ|ፀ) { printf("First Order (ግእዝ): %s\n", yytext); }
(ሁ|ሉ|ሑ|ሙ|ሩ|ሱ|ቁ|ቡ|ቱ|ኑ|ኡ|ኩ|ዉ|ዙ|ዱ|ጉ|ጡ|ፁ) { printf("Second Order (ካእት): %s\n", yytext); }
(ሂ|ሊ|ሒ|ሚ|ሲ|ቂ|ቢ|ቲ|ኒ|ኢ|ኪ|ዊ|ዚ|ዲ|ጊ|ጢ|ፂ) { printf("Third Order (ሳልስ): %s\n", yytext); }
(ሃ|ላ|ሓ|ማ|ሳ|ቃ|ባ|ታ|ና|ኣ|ካ|ዋ|ዛ|ዳ|ጋ|ጣ|ፃ) { printf("Fourth Order (ራብዕ): %s\n", yytext); }
(ሄ|ሌ|ሔ|ሜ|ሴ|ቄ|ቤ|ቴ|ኔ|ኤ|ኬ|ዌ|ዜ|ዴ|ጌ|ጤ|ፄ) { printf("Fifth Order (ኃምስ): %s\n", yytext); }
(ህ|ል|ሕ|ም|ስ|ቅ|ብ|ት|ን|እ|ክ|ው|ዝ|ድ|ግ|ጥ|ፅ) { printf("Sixth Order (ሳድስ): %s\n", yytext); }
(ሆ|ሎ|ሖ|ሞ|ሶ|ቆ|ቦ|ቶ|ኞ|ኦ|ኮ|ዎ|ዞ|ዶ|ጎ|ጦ|ፆ) { printf("Seventh Order (ሰብዕ): %s\n", yytext); }

.|\n { /* Ignore other characters */ }

%%

int main() {
    yylex();
    return 0;
}

int yywrap() {
    return 1;
}


Writing fidel_orders.l


In [ ]:
!flex fidel_orders.l
!gcc lex.yy.c -o fidel_orders

In [ ]:
!echo "ሀ ሁ ሂ ሃ ሄ ህ ሆ መ ሙ ሚ ማ ሜ ም ሞ" | ./fidel_orders

First Order (ግእዝ): ሀ
Second Order (ካእት): ሁ
Third Order (ሳልስ): ሂ
Fourth Order (ራብዕ): ሃ
Fifth Order (ኃምስ): ሄ
Sixth Order (ሳድስ): ህ
Seventh Order (ሰብዕ): ሆ
First Order (ግእዝ): መ
Second Order (ካእት): ሙ
Third Order (ሳልስ): ሚ
Fourth Order (ራብዕ): ማ
Fifth Order (ኃምስ): ሜ
Sixth Order (ሳድስ): ም
Seventh Order (ሰብዕ): ሞ


In [ ]:
%%writefile longest_word.l
%{
#include <stdio.h>
#include <string.h>

int max_length = 0;
char longest_word[100]; // Store the longest word
%}

%%

[a-zA-Z]+ {
    int len = strlen(yytext);
    if (len > max_length) {
        max_length = len;
        strcpy(longest_word, yytext);
    }
}

[^a-zA-Z]+  ; // Ignore non-word characters (spaces, punctuation)

%%

int main() {
    yylex();
    printf("Longest Word: %s\n", longest_word);
    printf("Length: %d\n", max_length);
    return 0;
}

int yywrap() {
    return 1;
}


Writing longest_word.l


In [ ]:
!flex longest_word.l
!gcc lex.yy.c -o longest_word

In [ ]:
!echo "Lex is a lexical analyzer generator for building programs" | ./longest_word

Longest Word: generator
Length: 9


In [ ]:
%%writefile count_numbers.l
%{
#include <stdio.h>

int positive_count = 0, negative_count = 0;
%}

%%

-?[0-9]+ {
    if (yytext[0] == '-') {
        negative_count++;
        printf("Negative Number: %s\n", yytext);
    } else {
        positive_count++;
        printf("Positive Number: %s\n", yytext);
    }
}

.|\n { /* Ignore non-numeric characters */ }

%%

int main() {
    yylex();
    printf("\nTotal Positive Numbers: %d\n", positive_count);
    printf("Total Negative Numbers: %d\n", negative_count);
    return 0;
}

int yywrap() {
    return 1;
}


Writing count_numbers.l


In [ ]:
!flex count_numbers.l
!gcc lex.yy.c -o count_numbers

In [ ]:
!echo "45 -12 0 89 -56 -100 200" | ./count_numbers

Positive Number: 45
Negative Number: -12
Positive Number: 0
Positive Number: 89
Negative Number: -56
Negative Number: -100
Positive Number: 200

Total Positive Numbers: 4
Total Negative Numbers: 3


In [51]:
%%writefile README.md
# LEX Programs Submission
**Name:** Bethlehem Yalelet
**ID:** 145614



[Open in Colab](https://colab.research.google.com/drive/1tCdMHcpVSG7TZE97wRzdUdSueyNBFXBD?usp=sharing)

Overwriting README.md
